In [1]:
import torch
from data.shapenet import ShapeNet
from model.vertix_model import VertixModel
import tqdm
from util.visualization import visualize_pointcloud, visualize_mesh
import random
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment
from sklearn.utils import class_weight

In [2]:
num_vertices = 100

In [3]:
config = {
    'experiment_name': 'vertix_edge_overfitting',
    'device': 'cuda:0',  
    'is_overfit': True,
    'batch_size': 8,
    'resume_ckpt': False,
    'learning_rate': 5e-4,
    'max_epochs': 50000,
    'print_every_n': 10,
    'validate_every_n': 50,
    'sdf_path': 'data/shapenet_dim32_sdf',
    'meshes_path': 'data/shapenet_reduced',
    'class_mapping': 'data/shape_info.json',
    'split': 'overfit',
    'num_vertices': num_vertices,
    'feature_size': 512,
    'num_trajectories': 2,
}

In [4]:
dataset = ShapeNet(sdf_path=config["sdf_path"],
                         meshes_path=config["meshes_path"],
                         class_mapping=config["class_mapping"],
                         split = "val", threshold=config["num_vertices"], num_trajectories=config["num_trajectories"])


In [5]:
dataset.filter_data()

Length of dataset: 32304
Filtering data ..


100%|███████████████████████████████████| 32304/32304 [00:06<00:00, 5348.71it/s]

Length of dataset: 5474


In [6]:
from inference.inference_vertix_edge import InferenceHandlerVertixEdgeModel

inferer = InferenceHandlerVertixEdgeModel('runs/vertix_edge_hungarian_train/model_best.ckpt', config["num_vertices"], config["feature_size"])

In [7]:
x_indices = []
y_indices = []

graph = np.ones((1,config["num_vertices"], config["num_vertices"],1))

for i in range(config["num_vertices"]):
    for j in range(config["num_vertices"]):
        x_indices.append(i)
        y_indices.append(j)

In [10]:
def visualize_random():
    random_sample = random.randint(0,len(dataset)-1)
    input_sdf, target_vertices, mask, target_edges, edges_adj = dataset[random_sample]
    faces = []

    for i in range(num_vertices):
        for j in range(num_vertices):
            for k in range(num_vertices):
                if i != j and j != k and k != i:
                    if target_edges[i][j] ==1 and target_edges[j][k]==1 and target_edges[k][i]==1:
                        faces.append(np.array([i,j,k]).reshape(1,-1))
                    
    faces = np.concatenate(faces,0)
    
    

                    
        
    print("GT Mesh")
    
    
    visualize_mesh(target_vertices, faces)
    
    output_pointcloud, edges = inferer.infer_single(input_sdf, mask, x_indices,y_indices,edges_adj)
    
    faces = []
    
    
    print("Predicted Pointcloud")
    
    visualize_pointcloud(output_pointcloud,point_size=0.01)

    
    target_size = int(sum(mask))
    
    matched_edges = np.zeros((num_vertices,num_vertices))
                    
    cost = distance_matrix(output_pointcloud, target_vertices[:int(sum(mask))])
                        
    vertix_idx, target_idx = linear_sum_assignment(cost) 
    
    for i in range(target_size):
        for j in range(target_size):
            curr_v_1 = vertix_idx[i]
            curr_t_1 = target_idx[i]
            curr_v_2 = vertix_idx[j]
            curr_t_2 = target_idx[j]
            matched_edges[curr_v_1,curr_v_2] = target_edges[curr_t_1,curr_t_2]
            
    faces = []

    for i in range(num_vertices):
        for j in range(num_vertices):
            for k in range(num_vertices):
                if edges[i][j] and edges[j][k] and edges[k][i]:
                    faces.append(np.array([i,j,k]).reshape(1,-1))
    faces = np.concatenate(faces,0)
    

    print("Predicted Mesh")
    
    visualize_mesh(output_pointcloud,faces)
    

In [11]:
visualize_random()

GT Mesh


Output()

Predicted Pointcloud


Output()

Predicted Mesh


Output()

Output()

NameError: name 'algorithm' is not defined